In [98]:
import pandas as pd
import numpy as np
import math

In [6]:
chess = pd.read_csv('../data/caissabase_df.csv', index_col = 0)

C:\Users\apcsm\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9,12,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\apcsm\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [11]:
chess.loc[chess['result'] == '1-0', 'white_perf'] = chess['black_elo'] + 400 
chess.loc[chess['result'] == '1/2-1/2', 'white_perf'] = chess['black_elo']
chess.loc[chess['result'] == '0-1', 'white_perf'] = chess['black_elo'] - 400

In [17]:
chess.loc[chess['result'] == '1-0', 'black_perf'] = chess['white_elo'] - 400 
chess.loc[chess['result'] == '1/2-1/2', 'black_perf'] = chess['white_elo']
chess.loc[chess['result'] == '0-1', 'black_perf'] = chess['white_elo'] + 400

In [19]:
chess['white_perf_diff'] = chess['white_perf']-chess['white_elo']
chess['black_perf_diff'] = chess['black_perf']-chess['black_elo']

In [28]:
white_first_move_adv = chess['white_perf_diff'].mean()

In [32]:
chess['white_perf_diff_adjusted'] = chess['white_perf_diff']-white_first_move_adv
chess['black_perf_diff_adjusted'] = chess['black_perf_diff']+white_first_move_adv

In [53]:
chess = chess.astype({'white_perf':'int', 'black_perf':'int', 'white_perf_diff':'int', 'black_perf_diff':'int', 'white_perf_diff_adjusted':'int', 'black_perf_diff_adjusted':'int'})

In [143]:
chess

,event,white_elo,black_elo,result,site,date,white,white_fide_id,white_title,black,black_fide_id,black_title,eco,event_date,opening,variation,1w,1b,2w,2b,3w,3b,4w,4b,5w,5b,6w,6b,7w,7b,8w,8b,9w,9b,10w,10b,11w,11b,12w,12b,13w,13b,14w,14b,15w,15b,16w,16b,17w,17b,18w,18b,19w,19b,20w,20b,white_perf,black_perf,white_perf_diff,black_perf_diff,white_perf_diff_adjusted,black_perf_diff_adjusted
0,Tel Aviv Int. 2021,2337,2251,1/2-1/2,Tel Aviv ISR,2021-11-29,"Solomon, Asif",2820218.0,FM,"Kaganskiy, G",4132025.0,FM,E46,2021.11.28,Nimzo-Indian,4.e3 O-O,d4,Nf6,c4,e6,Nc3,Bb4,e3,O-O,Bd2,b6,Nf3,Bb7,Bd3,d6,O-O,Nbd7,b3,e5,Bc2,Re8,a3,Bxc3,Bxc3,e4,Nd2,d5,Qe2,Qe7,Bb2,c5,cxd5,cxd4,Bxd4,Bxd5,f4,Rac8,Bd1,Nc5,b4,Nd3,2251,2337,-86,86,-117,117
1,ch-NED Final 2021,2608,2450,1-0,Rotterdam NED,2021-11-29,"Warmerdam, Max",1048104.0,GM,"Lai, Hing Ting",1038257.0,IM,A18,2021.11.27,English,"Mikenas-Carls, Flohr variation",c4,Nf6,Nc3,e6,e4,d5,e5,d4,exf6,dxc3,bxc3,Qxf6,Nf3,Nc6,d4,e5,d5,Nb8,Bd3,Nd7,Bg5,Qd6,O-O,Be7,Bxe7,Qxe7,Re1,O-O,Bc2,Qf6,Ba4,c6,Rb1,h6,Bc2,Re8,Qd3,g6,Nd2,Kg7,2850,2208,242,-242,210,-210
2,Tel Aviv Int. 2021,2299,2517,1/2-1/2,Tel Aviv ISR,2021-11-29,"Aizenberg, Benny",2821125.0,FM,"Rozentalis, E",12800023.0,GM,C60r,2021.11.28,Ruy Lopez,Cozio defence,e4,e5,Nf3,Nc6,Bb5,Nge7,c3,d6,d4,Bd7,O-O,Ng6,d5,Nb8,Bxd7+,Nxd7,c4,Be7,Nc3,O-O,Be3,f5,exf5,Rxf5,b4,a5,a3,Qf8,Nd2,Nf4,Nde4,Qf7,Ng3,Rg5,Nb5,Rc8,bxa5,h5,Bxf4,exf4,2517,2299,218,-218,186,-186
3,Tel Aviv Int. 2021,2412,2297,1/2-1/2,Tel Aviv ISR,2021-11-29,"Kantsler, B",2805359.0,GM,"Zemach, Roee",2818531.0,FM,E54,2021.11.28,Nimzo-Indian,"4.e3, Gligoric system with 7...dc",d4,Nf6,c4,e6,Nc3,Bb4,e3,O-O,Bd3,d5,Nf3,dxc4,Bxc4,c5,O-O,cxd4,exd4,b6,Bg5,Bb7,Rc1,h6,Bh4,Nc6,a3,Be7,Ba2,Rc8,Re1,Nh5,Bg3,Nxg3,hxg3,Bf6,d5,exd5,Bxd5,Na5,Bxb7,Nxb7,2297,2412,-115,115,-146,146
4,ch-NED Final 2021,2570,2523,1/2-1/2,Rotterdam NED,2021-11-29,"Pruijssers, R",1013068.0,GM,"Swinkels, R",1010824.0,GM,B67c,2021.11.27,Sicilian,"Richter-Rauzer, Rauzer attack, 7...a6 defence, 8...Bd7",e4,c5,Nc3,Nc6,Nge2,d6,d4,cxd4,Nxd4,Nf6,Bg5,e6,Qd2,a6,O-O-O,Bd7,f3,Nxd4,Qxd4,Be7,g4,h5,gxh5,Nxh5,Bxe7,Qxe7,Rg1,Bc6,Qxd6,Qxd6,Rxd6,Nf4,Rd2,g6,b4,Ke7,a4,Rh5,Kb2,Rah8,2523,2570,-47,47,-78,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850244,Capodanno,2373,2544,0-1,Reggio Emilia ITA,2000-01-01,"Manca, Federico",NaN,NaN,"Solozhenkin, Evgeniy",NaN,NaN,B19b,NaN,NaN,NaN,e4,c6,d4,d5,Nc3,dxe4,Nxe4,Bf5,Ng3,Bg6,h4,h6,Nf3,e6,Ne5,Bh7,Bd3,Bxd3,Qxd3,Nd7,f4,c5,Bd2,Nxe5,fxe5,Qxd4,Qe2,Ne7,O-O-O,Nc6,Rhe1,Qa4,Kb1,O-O-O,b3,Qxh4,Ne4,Rd5,Bc3,Qd8,2144,2773,-229,229,-260,260
1850245,Hilton Open,2131,2458,0-1,Basel SUI,2000-01-01,"Speck, Hans",NaN,NaN,"Berzinsh, Roland",NaN,NaN,C44k,NaN,NaN,NaN,e4,e5,Nf3,Nc6,c3,d5,Bb5,dxe4,Nxe5,Qd5,Qa4,Ne7,Nxc6,bxc6,Bc4,Qg5,Qb3,Nd5,g3,Bd6,O-O,Nf4,d4,Nh3+,Kg2,Qh5,f4,exf3+,Rxf3,O-O,Be3,Ng5,Bxg5,Bh3+,Kf2,Qxg5,Qc2,Bg4,h4,Qh6,2058,2531,-73,73,-104,104
1850246,Hambourg IHEM,2258,2339,1-0,Hambourg,2000-01-01,"Voigt, Martin",NaN,NaN,"Lindinger, Markus",NaN,NaN,B33n,2000.05.29,NaN,NaN,e4,c5,Nf3,Nc6,d4,cxd4,Nxd4,Nf6,Nc3,e5,Ndb5,d6,Bg5,a6,Na3,b5,Nd5,Be7,Bxf6,Bxf6,c3,Ne7,Nc2,Nxd5,exd5,O-O,h4,e4,g3,Be5,Qd2,f5,O-O-O,Bd7,Kb1,a5,Nd4,b4,cxb4,axb4,2739,1858,481,-481,449,-449
1850247,Hilton Open,2565,2330,1-0,Basel SUI,2000-01-01,"Sokolov, An",NaN,NaN,"Milosevic, G.",NaN,NaN,B43o,NaN,NaN,NaN,e4,c5,Nf3,e6,d4,cxd4,Nxd4,a6,Nc3,Qc7,Bd3,Nf6,O-O,d6,f4,Be7,Nf3,Nc6,Qe2,O-O,Kh1,Re8,e5,Nd7,Ng5,g6,Nxf7,Kxf7,f5,Kg8,fxg6,Rf8,gxh7+,Kh8,Qg4,NaN,NaN,NaN,NaN,NaN,2730,2165,165,-165,133,-133


In [170]:
w1 = chess.groupby(['1w']).agg(mean = ('white_perf_diff_adjusted', 'mean'), count = ('white_perf_diff_adjusted', 'count'), standard_deviation = ('white_perf_diff_adjusted', 'std'))
w1['95%_confidence_interval_+'] = w1['mean'] + 1.96*(w1['standard_deviation']/(np.sqrt(w1['count'])))
w1['95%_confidence_interval_-'] = w1['mean'] - 1.96*(w1['standard_deviation']/(np.sqrt(w1['count'])))
w1['+/-range'] = (w1['95%_confidence_interval_+']-w1['95%_confidence_interval_-'])/2
w1 = w1.reset_index()
w1

,1w,mean,count,standard_deviation,95%_confidence_interval_+,95%_confidence_interval_-,+/-range
0,Na3,-42.000000,15,257.936316,88.533786,-172.533786,130.533786
1,Nc3,-19.921842,2866,292.389007,-9.217031,-30.626654,10.704811
2,Nf3,-6.021839,186500,269.772641,-4.797464,-7.246214,1.224375
3,Nh3,-86.897436,39,251.558384,-7.945540,-165.849332,78.951896
4,a3,-27.474085,656,306.116741,-4.048458,-50.899713,23.425627
5,a4,-81.923077,13,249.882460,53.914575,-217.760729,135.837652
6,b3,-22.657389,8946,288.830136,-16.672114,-28.642664,5.985275
7,b4,-34.859571,1353,305.866689,-18.561382,-51.157761,16.298189
8,c3,-32.503247,308,296.761604,0.639494,-65.645988,33.142741
9,c4,-6.887200,122899,274.492595,-5.352539,-8.421861,1.534661


In [172]:
b1 = chess.groupby(['1w','1b']).agg(mean = ('white_perf_diff_adjusted', 'mean'), count = ('white_perf_diff_adjusted', 'count'), standard_deviation = ('white_perf_diff_adjusted', 'std'))
b1['95%_confidence_interval_+'] = b1['mean'] + 1.96*(b1['standard_deviation']/(np.sqrt(b1['count'])))
b1['95%_confidence_interval_-'] = b1['mean'] - 1.96*(b1['standard_deviation']/(np.sqrt(b1['count'])))
b1['+/-range'] = (b1['95%_confidence_interval_+']-b1['95%_confidence_interval_-'])/2
b1 = b1.reset_index()
b1

,1w,1b,mean,count,standard_deviation,95%_confidence_interval_+,95%_confidence_interval_-,+/-range
0,Na3,Nf6,-71.333333,3,118.938359,63.258091,-205.924757,134.591424
1,Na3,d5,-1.000000,4,194.999145,190.099162,-192.099162,191.099162
2,Na3,e5,-158.500000,4,269.795602,105.899690,-422.899690,264.399690
3,Na3,e6,328.000000,1,NaN,NaN,NaN,NaN
4,Na3,g6,-35.333333,3,430.158498,451.436831,-522.103497,486.770164
5,Nc3,Nc6,-175.611111,18,323.448721,-26.185426,-325.036797,149.425686
6,Nc3,Nf6,-8.352941,289,309.570637,27.338732,-44.044615,35.691673
7,Nc3,a6,298.333333,3,204.326536,529.550575,67.116091,231.217242
8,Nc3,b6,-135.444444,18,236.146321,-26.350410,-244.538479,109.094034
9,Nc3,c5,-30.886150,852,294.819210,-11.089484,-50.682816,19.796666


In [ ]:
#I am attempting to make a 'scoring' sytem that scores moves based on: 
# 1. count of times move has appeared (less is better).
# 2. Moves elo score perf (scores with low cofidence will be penalized).
# 3. % of previous position this move is played (lower is better).
# *Scores will only be calculated if count is 25 or higher to prevent a massive number of possibilites.
# I'm currently stuck on #3- how do I compare a move to the previous one? Time will tell. 

In [148]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)